In [1]:
#include <iostream>
#include <cmath>

In [2]:
#pragma cling add_include_path("/p/project/training2213/local/include")

## Functions

As the last topic for this notebook on fundamental concepts, let's examine functions. Unlike what we have been doing in this interpreted environment, all executable code like, `1 + 1`, `std::cout << whatever`, `for` loops, `if` statements ... all such code must be inside functions in C++. Only declarations of global variables, class and function definitions and other similar things can occur outside function bodies. In fact, function definitions **can not be inside bodies of other functions**.

Functions are recipes for calculating zero or more outputs from zero or more inputs.

```c++
auto func_name(double x) -> double
{
    if (x > 1.0 or x < -1.0) 
        return 0;
    else 
        return (1 - x) * (1 - x);
}
```

Any conceptually important input->output mapping can be made into a function. The top line in the function definition above, called its header, makes the input->output connections clear. Function inputs go in the parentheses `()`, and the output is written to the right of the input: `(inputs) -> output`. The older function syntax, shared with "C" is written as follows:
```c++
double func_name(double x)
{
    // recipe
}
```

This is still valid, and in this notebook it is the only working syntax for functions. But I consider the newer syntax with output to the right clearer and preferable. Outside this notebook, that syntax should be favoured.

### Side effects

Functions may also have side effects on the global state of the program.

```c++
auto largest_squared_num = 0.;
auto sqr(double x) -> double
{
    x = x * x;
    if (x > largest_squared_num)
        largest_squared_num = x;
    return x;
}
```

Every time we call the above function, there is a chance that we modify the global variable `largest_squared_num`. We can always look at the variable `largest_squared_num` to see the largest square calculated in the program so far. That value is not the "result" of the function, but it may change when we call `sqr`. That is called a side-effect. Any time you write something with `std::cout`, it is a side effect, because it changes the state of the global IO buffers. Side effects are sometimes necessary, or else we will not be able to write anything on the screen or in a file. Functions without any side-effects, strictly calculating an output from some inputs, are called "pure" functions. `func_name` above is a pure function. `sqr` above is not. Pure functions can be called simultaneously from any number of threads, and we can be confident that each of the calls will produce the right answer. If a function modifies global state, those modifications must be carefully orchestrated so that, multiple threads do not simultaneously modify a global variable, which can lead to incorrect or invalid states for those global variables.

Using global constants does not modify global state, and hence does not have side effects.

When a function is not meant to calculate and return any answer, its return type is written as `void`. Since `auto func(int i) -> void` looks somewhat awkward, we use the older syntax `void fun(int i)` for those functions. It helps distinguishing those "procedures" from other functions. A `void` returning function which is also pure, is simply some extra text. (no effects, no side-effects)

If the formal parameter is an L-value reference, the function may change the value of the object used to initialize it. This is another kind of side effect. L-value reference parameters to a function can be considered **out**-parameters, or **in-out**-parameters.

### Variable visibility

The function bodies define a top level block scope. The only variables known in a function are

    - those which are declared in the function
    - input parameters
    - global (namespace) scope variables
    
In particular, variables in the bodies of other functions are not available for use. The scoping rules, different control flow mechanisms you have learned so far can be used to write the recipe to calculate output values from inputs.

Let's practice that!

### Exercise

Write a function taking the distance between two particles and their masses as inputs, producing the Newtonian gravitational potential energy between them as the output. The Newtonian gravitational potential is given by $V(m_1, m_2, r) = -G\frac{m_1 m_2}{r}$.

### Exercise

We have a system of N particles so that their x, y, z positions and their masses are stored in `std::vector`s. Write a function to calculate the total gravitational potential energy of the system.

### Mental model for function calls

To write good functions, one has to understand a little bit about what happens when we use (call) a function. Whenever we call a function, the recipe in the function body is executed. Let's illustrate by taking an example function where we add up the elements of an input `vector`:

```c++
auto addup(Something V) -> double
{
    double ans{};
    for (auto&& elem: V) ans += elem;
    return ans;
}

auto elsewhere()
{
    std::vector masses{1., 1., 1.2, 1.4, 0.9, 1.5, 1.1 };
    std::vector charges{1.0, 1.0, 0.1, 0.1, 0.1, 0.1, 0.1};
    std::cout << "Total mass = " << addup(masses) << "\n";
    std::cout << "Total charge = " << addup(masses) << "\n";
}

```

What happens in the line where we call `addup(masses)` ? Somehow the recipe for adding up the elements must be executed with the elements of the vector `masses` and not the vector `charges`. Similarly when we call `addup(charges)` the exact same recipe will be executed, but now `V` representing the `charges` vector. 

The variable `V` in the function header is called, in other languages, a formal argument or a dummy argument. Before the recipe of the function is executed, the formal arguments are initialised using the corresponding values in the function call expression. Here is the crucial point: **this initialisation is exactly like a variable initialisation from an expression!** We are essentially replacing `addup(masses)` by the result of the following procedure:

```c++
Something V{masses};
double ans{};
for (auto&& elem: V) ans += elem;
result = ans;
```

This is why we spent such a long time discussing name stickers, duplicates and so on. Remember how we created duplicates ? To create a dupliate `vector` from a `vector`, you would write:

```c++
std::vector<double> duplicate{original};
```

If we substitute `Something` above with `std::vector<double>`, our pseudo-code recipe above would basically create a duplicate vector before adding its elements. Do we really need a duplicate vector to add the elements ?

We learned how to create aliases. `std::vector<double>& alias{original};`. Aliases or references are extra name stickers attached to the same objects. If we substitute `Something` with `std::vector<double>&`, our pseudo-code recipe becomes,

```c++
std::vector<double>& V{masses};
double ans{};
for (auto&& elem: V) ans += elem;
result = ans;
```

`V` becomes another name for `masses` when we are evaluating `addup(masses)`, and for `charges` when we are evaluating `addup(charges)`! This is good. But we can do better! Since the purpose of the `addup` function is to give us the sum, it makes no sense that it should change the `masses` or `charges`. We can make the `V` a constant reference, i.e., a read-only alias, so that we can be sure, that the function does not tamper with the masses. Thus,

```c++
auto addup(const std::vector<double>& V) -> double
{
    double ans{};
    for (auto&& elem: V) ans += elem;
    return ans;
}
```


### Quiz:

Consider the very simple function below. Why does one of the attempts to use it work but the next one does not ?

In [3]:
int f1(int& i)
{
    return i + 2;
}

In [4]:
{
    int N{4000};
    std::cout << f1(N) << "\n"; // this works
    // std::cout << f1(4000) << "\n"; // this does not. Why ?
}

4002


It is always possible to understand what works and does not work based on the mental picture discussed earlier. Function call involves initialisation of the formal function parameters using the call expression. We can not initialise a non-constant L-value reference (`int&` above) from a pure R-value (`4000`). Since a constant reference can not be used on the left side of an assignment, we can create such entites from pure values like `4000`.  

In [5]:
const int& i{4000};

Therefore, a function declared in terms of a `const` reference can accept normal names as well as values as inputs.

In [6]:
int f2(const int&i)
{
    return i + 2;
}

In [7]:
{
    int N{4000};
    std::cout << f2(N) << "\n";
    std::cout << f2(4000) << "\n";
}

4002
4002


### Quiz:

Why does the `cout` in the loop below execute 10 times, even if we call the increment function in the loop ?

In [8]:
void increment(int i)
{
    i = i + 1;
}

In [9]:
for (auto i = 0; i < 10; ++i) {
    std::cout << i << "\n";
    increment(i);
}

0
1
2
3
4
5
6
7
8
9


Using our mental model of function calls, the `increment` function looks like this:

```c++
int i_duplicate{i_original};
i_duplicate = i_duplicate + 1;
```

As you can see, we are never touching the `i_original`, from inside the loop, inside the function. So, that `i` only gets incremented in the `for` loop. 

`TypeName varname{initialier}` *always* creates a new object, whether the initializer is a pure value, like `5` or the name of another variable, like `i`. That's what we arranged for, when we wrote our function input parameter as `int i`. 

How would you change the increment function so that it does not create a duplicate, but works on the same object that we pass it when calling the function ? Write a **different function with a different name** and test! If you do not give the function a different name, we get into "ambiguous" definitions, and the fascinating topic of function overloading in C++ (next section).

## Function overloading

It is possible to have multiple functions of the same human readable name, as long as they can be distinguished by the types of the input parameters.

```c++
auto power(double x, int i) -> double
{
    if (i == 0) return 1.;
    bool inv = (i < 0);
    auto ans = 1.;
    i = std::abs(i);
    for (auto j = 0; j < i; ++j) ans *= x;
    if (inv) ans = 1.0/ans;
    return ans;
}

auto power(double x, double y) -> double
{
    return std::exp(y * std::log(x));
}

void elsewhere()
{
    std::cout << power(3.14, 4) << "\n"; // calls first version
    std::cout << power(3.14, 4.) << "\n"; // calls second version
}
```

Let's see that in action, and discuss.

In [10]:
double power(double x, int i)
{
    std::cout << "Calculating integer power " << i << " of " << x << "\n";
    if (i == 0) return 1.;
    bool inv = (i < 0);
    auto ans = 1.;
    i = std::abs(i);
    for (auto j = 0; j < i; ++j) ans *= x;
    if (inv) ans = 1.0/ans;
    return ans;
}


In [11]:
double power(double x, double y)
{
    std::cout << "Calculating floating point power " << y << " of " << x << "\n";
    if (x < 0.) throw std::runtime_error{"Bad negative input for non-integral power." };
    return std::exp(y * std::log(x));
}

In [12]:
power(3.14, 4.0);

Calculating floating point power 4 of 3.14


(double) 97.211712


In [13]:
power(3.14, 4);

Calculating integer power 4 of 3.14


(double) 97.211712


We didn't have to invent different names like `power_double_int` and `power_double_double` for these functions. This is because that is conceptually what the compiler is doing for us! To the C++ compiler, the name of a function is not just the name by which you see it, but it is that name + the types of all the inputs to the function. Therefore, as far as the compiler is concerned, we declared two completely different functions above. How does the compiler know which one to invoke when we write an expression like `power(a, b)` ?

Even at the call site, if `a` is a double and `b` is an integer, the compiler sees it as an attempt to call `power_double_int`. The existence of the other function does not matter. If `b` is a double, it is again, clear to the compiler! You want to call `power_double_double`, and the compiler knows what that is!

`power` above is said to be an overloaded function with two overloads. The collection of all functions sharing the same human readable name and different input parameter types is called an "overload set".

How much time does this kind of overloading cost when the program is running ?

<h1>0.0 femtoseconds!</h1>

The function overloading mechanism is a purely compile time process. In any C++ program, the compiler resolves the entire possible set of function calls ahead of time (Let's discuss *virtual* function calls in due time along with C++ classes). This decision, whether to call one or the other version of `power` is not happening at program execution time.

It is important that the compiler is able to distinguish between the overloads. You can not have two versions of `power` each taking a `double` and `int`, but one handling positive integers and the other handling negative integrs. You might think that `double` and `int` are, of course, easy to tell apart. How similar can the type of the second input parameter be and still be used to overload `power` ?

In [14]:
double power(double x, short i)
{
    std::cout << "A possible third implementation of the same function.\n";
    return 0.; // We are just focussing on overload resolution right now.
}

In [15]:
power(3.14, 2);

Calculating integer power 2 of 3.14


(double) 9.8596000


In [16]:
power(3.14, short{2}); // There is no literal suffix for short.

A possible third implementation of the same function.


(double) 0.0000000


In [17]:
double power(double x, long i)
{
    std::cout << "double to the power long\n";
    return 0.; // Just focussing on overload resolution...
}

In [18]:
power(3.14, 2L); // There is a literal suffix for long

double to the power long


(double) 0.0000000


In [19]:
double power(double x, unsigned long i)
{
    std::cout << "double to the power unsigned long\n";
    return 0.; // Just focussing on overload resolution...
}

In [20]:
power(3.14, 2UL);

double to the power unsigned long


(double) 0.0000000


In [21]:
power(3.14, 2.);

Calculating floating point power 2 of 3.14


(double) 9.8596000


In [22]:
power(3.14, 8);

Calculating integer power 8 of 3.14


(double) 9450.1170


In [23]:
power(3.14, short{8});

A possible third implementation of the same function.


(double) 0.0000000


In [24]:
power(3.14, 8L);

double to the power long


(double) 0.0000000


In [25]:
char c{2};
power(3.14, c);

Calculating integer power 2 of 3.14


(double) 9.8596000


Above, we did not write a version of the overload for the `char` type. In this case, the compiler tries "integer promotion" to promote our `char` input into an `int` and uses the `int` version. All shorter than 32 bit integers are promoted to the "natural" 32 bit integers on current implementations.

In [26]:
double p{0.77};
power(3.14, p);

Calculating floating point power 0.77 of 3.14


(double) 2.4134362


Now let's examine overloading, but this time with functions taking references as formal parameters.

In [27]:
void expt(double& x)
{
    std::cout << "Version accepting mutable L-value reference\n";
}

In [28]:
void expt(const double& x)
{
    std::cout << "Version accepting constant L-value reference\n";
}

In [29]:
expt(0.);

Version accepting constant L-value reference


(void) @0x7f226d7f8dc0


In [30]:
double x = 9.8;

In [31]:
expt(x);

Version accepting mutable L-value reference


(void) nullptr


In [32]:
const double& y{x};
double& z{x};
std::cout << "Memory address of the object for x : " << (size_t)std::addressof(x) << "\n"; 
std::cout << "Memory address of the object for y : " << (size_t)std::addressof(y) << "\n"; 
std::cout << "Memory address of the object for z : " << (size_t)std::addressof(z) << "\n";
expt(x);
expt(y);
expt(z);
expt(z + y);

Memory address of the object for x : 139786145235008
Memory address of the object for y : 139786145235008
Memory address of the object for z : 139786145235008
Version accepting mutable L-value reference
Version accepting constant L-value reference
Version accepting mutable L-value reference
Version accepting constant L-value reference


(void) @0x7f226d7f8dc0


As you can see, we can have two different versions of the `expt` function, one accepting a normal, mutable, L-value reference, and another constant L-value reference. When we call the function, depending on the nature of the expression used to call the function, one or the other version is called. Notice that the identifiers `x`, `y` and `z` all identify the same entity in the computer's memory. But one of them, `y` is a read-only identifier. When calling `expt`, the version invoked is based on whether we have a mutable identifier or an immutable entity in the argument at the call site.

In [33]:
void expt2(double& x)
{
    std::cout << "Version accepting mutable L-value reference\n";
}

In [34]:
void expt2(const double& x)
{
    std::cout << "Version accepting constant L-value reference\n";
}

In [35]:
void expt2(double x)
{
    std::cout << "Version where the formal argument is a value\n";
}

In [36]:
expt2(3.0);

input_line_39:2:2: error: call to 'expt2' is ambiguous
 expt2(3.0);
 ^~~~~
input_line_37:1:6: note: candidate function
void expt2(const double& x)
     ^
input_line_38:1:6: note: candidate function
void expt2(double x)
     ^


ename: evalue

In [37]:
expt2(x);

input_line_40:2:2: error: call to 'expt2' is ambiguous
 expt2(x);
 ^~~~~
input_line_36:1:6: note: candidate function
void expt2(double& x)
     ^
input_line_37:1:6: note: candidate function
void expt2(const double& x)
     ^
input_line_38:1:6: note: candidate function
void expt2(double x)
     ^


ename: evalue

As you can see, having a version with a value and another with a reference leads to ambiguities. The compiler can not distinguish between a pure value and a constant reference at the call site. Avoid overloading between plain types and references.

## Recursion

Remember the self-referencial definition of the factorial function in mathematics ? 

$$
N! = \left\{
    \begin{array}\\
        N\times (N-1)! & \mbox{if } \ N > \mathbf{1} \\
        1 & \mbox{if } \ N = \mathbf{0}\,or\,\mathbf{1} 
    \end{array}
\right.
$$

The exact same thing can be expressed in a C++ function (this is a syntax explanation. I am not recommending writing factorial this way!).

```c++
auto factorial(unsigned int N) -> unsigned int
{
    return (N > 1U) ? N * factorial(N - 1U) : 1U;
}
```
We used the ternary operator to compactly express the following:

```c++
if (N > 1U)
    return N * factorial(N - 1U);
else
    return 1U;
```

This type of expression, when a function calls itself from its own body, is called recursion. Recursive functions can sometimes be used to express complex tasks, such as tree processing, elegantly and compactly. But they also illustrate an important point: local variables as well as formal arguments of a function are private to a particular call. I refer you to the section 2.2.4.1 ("Whiteboard stack") for a more extended discussion. Right now, let's just rig our factorial function so that we can see this: 

In [38]:
unsigned int factorial(unsigned int N)
{
    std::cout << "Evaluating factorial of " << N << "\tParameter stored at address " << (size_t)(&N) <<"\n";
    if (N <= 1U) {
        std::cout << "No need for recursive calls. Returning 1\n";
        return 1U;
    } else {
        std::cout << "Starting recursive call from level " << N << "\n";
        unsigned int rest = factorial(N-1);
        std::cout << "At level N = " << N << ", with factorial of the rest = " 
            << rest << " stored at " << &rest << "\n";
        return N * rest;
    }
}

In [39]:
factorial(4);

Evaluating factorial of 4	Parameter stored at address 139785842691624
Starting recursive call from level 4
Evaluating factorial of 3	Parameter stored at address 139785842691576
Starting recursive call from level 3
Evaluating factorial of 2	Parameter stored at address 139785842691528
Starting recursive call from level 2
Evaluating factorial of 1	Parameter stored at address 139785842691480
No need for recursive calls. Returning 1
At level N = 2, with factorial of the rest = 1 stored at 0x7f226d7f8dc4
At level N = 3, with factorial of the rest = 2 stored at 0x7f226d7f8df4
At level N = 4, with factorial of the rest = 6 stored at 0x7f226d7f8e24


(unsigned int) 24


Notice how for each level of recursion, the parameter values were stored at different locations. Each **call** to a function has its own separate working area to store its formal parameters, local variables etc. This area is called the "stack frame" of a function. When a function returns, its stack frame is reused for subsequent function calls. This is why there is no danger that the local variable `rest` for one level of the factorial will be erased or overwirtten by the calculation from another level.

## Multiple output values

Syntactically, a C++ function has a single return type. There can be multiple return statements as in the `factorial` function above, but they must all return the same type of object. What if we need to return more than one output from a function ? There are many ways of doing that:

    - We can use non-constant L-value reference parameters, and assign to those parameters in the function
    - The function can return a `tuple` containing those different types
    - Multiple outputs can be bundled together in a `struct` or `class`
    
Let's illustrate that using a simple integer division function. We divide two integer inputs. The answer we seek are both quotient and remainder. 

In the first approach, our function could look like this:

```c++
auto integer_division(int L, int R, int& q, int& r) -> int
{
    q = L / R;
    r = L % R;
    return q;
}
```
This approach has been in use for a long time. To use this, we have to do something like this:

```c++
void elsewhere()
{
    int num1{somevalue}, num2{anothervalue};
    int quotvar, remvar;
    integer_division(num1, num2, quotvar, remvar);
    std::cout << "Quotient = " << quotvar << ", remainder = " << remvar << "\n";
}
```

The L-value reference parameters ensure that the assignments to `q` and `r` done inside the function are performed on `quotvar` and `remvar`. It works. But it obscures the distinction between the inputs and outputs for the function. It is easy to get confused about which arguments are inputs and which are outputs. The second approach looks like this:

```c++
auto integer_division(int L, int R) -> std::tuple<int, int>
{
    return std::make_tuple(L / R, L % R);
}

void elsewhere()
{
    int num1{somevalue}, num2{anothervalue};
    auto [quotvar, remvar] = integer_division(num1, num2);
    std::cout << "Quotient = " << quotvar << ", remainder = " << remvar << "\n";
}
```

There is no confusion regarding the inputs and outputs for this function. The output of the function is a tuple, but we can kind of "unpack" it using the notation shown. If we did not use the structured binding notation of `auto [name1, name2 ...] = tuple`, we will receive a tuple as the output. One can access the components of the tuple, but the notation is not as elegant:

```c++
void elsewhere()
{
    int num1{somevalue}, num2{anothervalue};
    auto res = integer_division(num1, num2);
    std::cout << "Quotient = " << std::get<0>(res) << ", remainder = " << std::get<1>(res) << "\n";
}
```

The third approach needs us to define a `struct` to staple together the values we want to receive as output.

```c++
struct div_result_type
{
    int quotient;
    int remainder;
};
```

`div_result_type` then becomes a new data type. One can create variables of this type. Each of those variables will have a "member" called quotient and another member called remainder. The members can be accessed using a dot notation :
```c++
div_result_type res{v1, v2};
// ...
if (res.quotient > res.remainder)
    do_something();
```

Our `integer_division` function can then be written as:

```c++
auto integer_division(int L, int R) -> div_result_type
{
    return { L / R, L % R };
}

void elsewhere()
{
    int num1{somevalue}, num2{anothervalue};
    auto res = integer_division(num1, num2);
    std::cout << "Quotient = " << res.quotient << ", remainder = " << res.remainder << "\n";
    // alternatively
    auto [q, r] = integer_division(num1, num2);
    std::cout << "Quotient = " << q << ", remainder = " << r << "\n";
}

```

This third approach involves one extra step when writing the function: creation of the `struct` representing the output type. It can be used with a structured binding declaration like a `tuple`. But there is a subtle advantage. With the tuple, we have to look at the code to see which component is the quotient and which one is the remainder. The fields of a tuple are simply "first field", "second field" ... They have no names. The implementer of the function can not indicate their intent about which field is what. One has to rely on "documentation": one of the most error prone ways known to mankind to determine the purpose of tuple components! In the `struct` approach, there can be descriptive names, so that the meaning of the fields is clear.

